In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import theano
import theano.tensor as tt
import arviz as az
import pandas as pd
from theano import shared
from scipy import stats
import math

# Helper functions

In [2]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def normalize(x):
    return x/np.sum(x)

def rmse(pred, actual):
    return math.sqrt(np.mean((pred-actual)**2))

def sort_category(unique, count):
    output = np.ones(3) - 1
    for i in range(3):
        for j in range(len(unique)):
            if(i==unique[j]):
                output[i]=count[j]
    return normalize(output) 

def perturb(x, noise=.001):
    return tt.switch(tt.ge(x,1),1-noise, tt.switch(tt.le(x,0), noise, x))

def compute_ci_discrete(x, HDI, n, sim):
    output = [None]*sim
    for i in range(sim):
        draw = np.random.choice(x, size = n, replace = True)
        unique, counts = np.unique(draw, return_counts=True)
        output[i] = sort_category(unique, counts)
    lower = np.ones(len(output[0]))
    upper = np.ones(len(output[0]))
    output = np.transpose(output) # each element is now all the samples of one category
    for i in range(len(output)):
        lower[i] = np.quantile(output[i], q = (1-HDI)/2)
        upper[i] = np.quantile(output[i], q = 1-(1-HDI)/2)
    return [lower, upper]

def compute_ci_continuous(x, HDI, n, sim):
    output = np.ones(sim)
    for i in range(sim):
        draw = np.random.choice(x, size = n, replace = True)
        output[i] = np.mean(draw)
    lower = np.quantile(output, q = (1-HDI)/2)
    upper = np.quantile(output, q = 1-(1-HDI)/2)
    return [lower, upper]

def computeA(m, s):
    return (m*s)/(1-m)

# Computational models for tracking study

## Data preprocessing

In [3]:
tracking_data = pd.read_csv('ecl_dat.csv', encoding="ISO-8859-1")
emotion_track = tracking_data.loc[tracking_data['condition'] == 'condition 1']
noemotion_track = tracking_data.loc[tracking_data['condition'] == 'condition 2']

### Computing means for causal inference (at various snapshots)

In [4]:
world1human_emotion_t1 = np.array(emotion_track["relation1BlueNorm"])
world2human_emotion_t1 = np.array(emotion_track["relation1OrangeNorm"])
world3human_emotion_t1 = np.array(emotion_track["relation1OrangeBlueNorm"])
worldhuman_emotion_t1 = sort_category([0,1,2],[np.mean(world1human_emotion_t1),np.mean(world2human_emotion_t1),np.mean(world3human_emotion_t1)])

world1human_noemotion_t1 = np.array(noemotion_track["relation1BlueNorm"]) 
world2human_noemotion_t1 = np.array(noemotion_track["relation1OrangeNorm"]) 
world3human_noemotion_t1 = np.array(noemotion_track["relation1OrangeBlueNorm"])
worldhuman_noemotion_t1 = sort_category([0,1,2],[np.mean(world1human_noemotion_t1),np.mean(world2human_noemotion_t1),np.mean(world3human_noemotion_t1)])

In [5]:
world1human_emotion_t2 = np.array(emotion_track["relation2BlueNorm"])
world2human_emotion_t2 = np.array(emotion_track["relation2OrangeNorm"])
world3human_emotion_t2 = np.array(emotion_track["relation2OrangeBlueNorm"])
worldhuman_emotion_t2 = sort_category([0,1,2],[np.mean(world1human_emotion_t2),np.mean(world2human_emotion_t2),np.mean(world3human_emotion_t2)])

world1human_noemotion_t2 = np.array(noemotion_track["relation2BlueNorm"]) 
world2human_noemotion_t2 = np.array(noemotion_track["relation2OrangeNorm"]) 
world3human_noemotion_t2 = np.array(noemotion_track["relation2OrangeBlueNorm"])
worldhuman_noemotion_t2 = sort_category([0,1,2],[np.mean(world1human_noemotion_t2),np.mean(world2human_noemotion_t2),np.mean(world3human_noemotion_t2)])

In [6]:
world1human_emotion_t3 = np.array(emotion_track["relation3BlueNorm"])
world2human_emotion_t3 = np.array(emotion_track["relation3OrangeNorm"])
world3human_emotion_t3 = np.array(emotion_track["relation3OrangeBlueNorm"])
worldhuman_emotion_t3 = sort_category([0,1,2],[np.mean(world1human_emotion_t3),np.mean(world2human_emotion_t3),np.mean(world3human_emotion_t3)])

world1human_noemotion_t3 = np.array(noemotion_track["relation3BlueNorm"]) 
world2human_noemotion_t3 = np.array(noemotion_track["relation3OrangeNorm"]) 
world3human_noemotion_t3 = np.array(noemotion_track["relation3OrangeBlueNorm"])
worldhuman_noemotion_t3 = sort_category([0,1,2],[np.mean(world1human_noemotion_t3),np.mean(world2human_noemotion_t3),np.mean(world3human_noemotion_t3)])

In [7]:
world1human_emotion_t4 = np.array(emotion_track["relation4BlueNorm"])
world2human_emotion_t4 = np.array(emotion_track["relation4OrangeNorm"])
world3human_emotion_t4 = np.array(emotion_track["relation4OrangeBlueNorm"])
worldhuman_emotion_t4 = sort_category([0,1,2],[np.mean(world1human_emotion_t4),np.mean(world2human_emotion_t4),np.mean(world3human_emotion_t4)])

world1human_noemotion_t4 = np.array(noemotion_track["relation4BlueNorm"]) 
world2human_noemotion_t4 = np.array(noemotion_track["relation4OrangeNorm"]) 
world3human_noemotion_t4 = np.array(noemotion_track["relation4OrangeBlueNorm"])
worldhuman_noemotion_t4 = sort_category([0,1,2],[np.mean(world1human_noemotion_t4),np.mean(world2human_noemotion_t4),np.mean(world3human_noemotion_t4)])

In [8]:
world1human_emotion_t5 = np.array(emotion_track["relation5BlueNorm"])
world2human_emotion_t5 = np.array(emotion_track["relation5OrangeNorm"])
world3human_emotion_t5 = np.array(emotion_track["relation5OrangeBlueNorm"])
worldhuman_emotion_t5 = sort_category([0,1,2],[np.mean(world1human_emotion_t5),np.mean(world2human_emotion_t5),np.mean(world3human_emotion_t5)])

world1human_noemotion_t5 = np.array(noemotion_track["relation5BlueNorm"]) 
world2human_noemotion_t5 = np.array(noemotion_track["relation5OrangeNorm"]) 
world3human_noemotion_t5 = np.array(noemotion_track["relation5OrangeBlueNorm"])
worldhuman_noemotion_t5 = sort_category([0,1,2],[np.mean(world1human_noemotion_t5),np.mean(world2human_noemotion_t5),np.mean(world3human_noemotion_t5)])

## Models (for the condition with no emotional display) across various snapshots

In [9]:
# These parameters are fixed across both conditions (emotions and no-emotions)
# A noise parameter which controls how much randomness is added at each conditional sampling (e.g., drawing actions from belief and desire) was allowed to vary across both conditions

# The model assumes that people perceive others to have high knowledgability.
# We simulated this using a beta distribution with a = 5 and b = 2 (mean of .71).
a = 5
b = 2

d = .5 # fixed prior for desire
delay = .2 # probability that first action had a delayed effect
human_prior = [0.3816152,0.3063088,0.3120760]

In [10]:
np.random.seed(123456)

In [11]:
with pm.Model() as model1ab:
    noise = .25
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both
    world_prior = pm.Categorical("world_prior", [human_prior], shape=(1)) # blue, orange, both
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    draw = 2000 
    trace1ab = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace1ab["world_prior"], return_counts=True)
worldposterior_t1 = sort_category(unique, counts)
rmse_output_t1 = rmse(worldposterior_t1, worldhuman_noemotion_t1)
print(rmse_output_t1)
unique, counts = np.unique(trace1ab["world"], return_counts=True)
worldposterior_t2 = sort_category(unique, counts)
rmse_output_t2 = rmse(worldposterior_t2, worldhuman_noemotion_t2)
print(rmse_output_t2)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief1_random, world_prior, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 4 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.008897303966385519
0.018941290048437084


In [12]:
with pm.Model() as model1c:
    noise = .25
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=2 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    draw = 2000 
    trace1c = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace1c["world"], return_counts=True)
worldposterior_t3 = sort_category(unique, counts)
rmse_output_t3 = rmse(worldposterior_t3, worldhuman_noemotion_t3)
print(rmse_output_t3)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 4 seconds.
The acceptance probability does not match the target. It is 0.8867651667344303, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


0.1805633883315242


In [13]:
with pm.Model() as model1d:
    noise = .25
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=2 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_blue_p = pm.Deterministic("action2_blue_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,0)*(1-noise) + \
                                                                  # if belief both are needed and orange was pushed previously, then push blue
                                                                  tt.eq(belief2,2)*tt.eq(action1,1)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,1)*noise,
                                                                  # if outcome was reached at t1, no further action required
                                                                  noise)) 
    action2_blue_p = perturb(action2_blue_p)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_orange_p = pm.Deterministic("action2_orange_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,1)*(1-noise) + \
                                                                  # if belief both are needed and blue was pushed previously, then push orange 
                                                                  tt.eq(belief2,2)*tt.eq(action1,0)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,0)*noise,
                                                                  noise)) 
    action2_orange_p = perturb(action2_orange_p)
    action2_stack = tt.transpose(tt.stack(action2_blue_p, action2_orange_p, [noise*2]))
    action2 = pm.Categorical("action2", action2_stack, observed=1, shape=(1)) # blue orange others
    
    draw = 2000 
    trace1d = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace1d["world"], return_counts=True)
worldposterior_t4 = sort_category(unique, counts)
rmse_output_t4 = rmse(worldposterior_t4, worldhuman_noemotion_t4)
print(rmse_output_t4)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 5 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.18072398840019893


In [14]:
with pm.Model() as model1:
    # the noise parameter affects the amount of randomness included into the sampling process. 
    # For example, a person's actions might sometimes be incongruent to his intentions and event relationships can be noisy
    # we set a higher noise parameter for the no-emotion condition as this led to slightly better model fit
    # however, even when we fixed the noise for both models, the overall inference patterns remained qualitatively similar
    noise = .25
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1)) # blue, orange, both
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire", 1, d, shape=(1)) # desire bulb to turn on=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p) # perturb ensures that the probability is not equals to 1 or 0
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if believe both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p) # perturb ensures that the probability is not equals to 1 or 0
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will likely not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome when probability is 1 or 0
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they likely won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=2 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_blue_p = pm.Deterministic("action2_blue_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,0)*(1-noise) + \
                                                                  # if believe both are needed and orange was pushed previously, then push blue
                                                                  tt.eq(belief2,2)*tt.eq(action1,1)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,1)*noise,
                                                                  # if outcome was reached at t1, no further action required
                                                                  noise)) 
    action2_blue_p = perturb(action2_blue_p)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_orange_p = pm.Deterministic("action2_orange_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,1)*(1-noise) + \
                                                                  # if believe both are needed and blue was pushed previously, then push orange 
                                                                  tt.eq(belief2,2)*tt.eq(action1,0)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,0)*noise,
                                                                  noise)) 
    action2_orange_p = perturb(action2_orange_p)
    action2_stack = tt.transpose(tt.stack(action2_blue_p, action2_orange_p, [noise*2]))
    action2 = pm.Categorical("action2", action2_stack, observed=1, shape=(1)) # blue orange others
    
    outcome2_p = pm.Deterministic("outcome2_p", tt.eq(world,0)*tt.eq(action2,0) + \
                                  # if previous action has a delayed effect
                                  tt.eq(world,0)*tt.eq(action1,0)*tt.neq(action2,0)*delay + \
                                  tt.eq(world,1)*tt.eq(action2,1) + \
                                  # if previous action has a delayed effect
                                  tt.eq(world,1)*tt.eq(action1,1)*tt.neq(action2,1)*delay + \
                                  tt.eq(world,2)*tt.eq(action1,0)*tt.eq(action2,1)+ \
                                  tt.eq(world,2)*tt.eq(action1,1)*tt.eq(action2,0)) 
    outcome2_p = perturb(outcome2_p, noise)
    outcome2 = pm.Binomial("outcome2", 1, outcome2_p, observed = 1, shape = (1))
    
    happy2 = outcome2*desire*(1-noise) + noise*(1-desire)
    frustrated2 = (1-outcome2)*desire*(1-noise) + noise*(1-desire)
    neutral2 = [1]
    emotion_array2 = tt.transpose(tt.stack(happy2, frustrated2, neutral2 ))
    expression2 = pm.Categorical("expression2",emotion_array2,observed=2 ) # happy, frustrated, neutral
    
    draw = 2000 
    trace1e = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace1e["world"], return_counts=True)
worldposterior_t5 = sort_category(unique, counts)
rmse_output_t5 = rmse(worldposterior_t5, worldhuman_noemotion_t5)
print(rmse_output_t5)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 6 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.10009608917965063


### Summary statistics

In [15]:
unique, counts = np.unique(trace1ab["world_prior"], return_counts=True)
print("world t1: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t1: {0: 3008, 1: 2481, 2: 2511}
[0.376    0.310125 0.313875]


In [16]:
np.random.seed(123456)
print("world t1 :" + str(compute_ci_discrete(flatten(trace1ab["world_prior"]), HDI = .95, n = 1000, sim = 1000)))

world t1 :[array([0.347, 0.283, 0.285]), array([0.406, 0.336, 0.343])]


In [17]:
unique, counts = np.unique(trace1ab["world"], return_counts=True)
print("world t2: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t2: {0: 3586, 1: 1841, 2: 2573}
[0.44825  0.230125 0.321625]


In [18]:
np.random.seed(123456)
print("world t2 :" + str(compute_ci_discrete(flatten(trace1ab["world"]), HDI = .95, n = 1000, sim = 1000)))

world t2 :[array([0.417975, 0.205   , 0.292   ]), array([0.479, 0.255, 0.352])]


In [19]:
unique, counts = np.unique(trace1c["world"], return_counts=True)
print("world t3: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t3: {0: 1288, 1: 2638, 2: 4074}
[0.161   0.32975 0.50925]


In [20]:
np.random.seed(123456)
print("world t3 :" + str(compute_ci_discrete(flatten(trace1c["world"]), HDI = .95, n = 1000, sim = 1000)))

world t3 :[array([0.139, 0.303, 0.48 ]), array([0.182, 0.357, 0.541])]


In [21]:
unique, counts = np.unique(trace1d["world"], return_counts=True)
print("world t4: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t4: {0: 801, 1: 2438, 2: 4761}
[0.100125 0.30475  0.595125]


In [22]:
np.random.seed(123456)
print("world t4 :" + str(compute_ci_discrete(flatten(trace1d["world"]), HDI = .95, n = 1000, sim = 1000)))

world t4 :[array([0.081, 0.275, 0.565]), array([0.12    , 0.333025, 0.624   ])]


In [23]:
unique, counts = np.unique(trace1e["world"], return_counts=True)
print("world t5: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t5: {0: 217, 1: 2589, 2: 5194}
[0.027125 0.323625 0.64925 ]


In [24]:
np.random.seed(123456)
print("world t5 :" + str(compute_ci_discrete(flatten(trace1e["world"]), HDI = .95, n = 1000, sim = 1000)))

world t5 :[array([0.017   , 0.296975, 0.621   ]), array([0.038   , 0.35    , 0.676025])]


## Models (for the condition with emotional displays) across various snapshots

In [25]:
np.random.seed(123456)

In [26]:
with pm.Model() as model2ab:
    noise = .15
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both
    world_prior = pm.Categorical("world_prior", [human_prior], shape=(1)) # blue, orange, both
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    draw = 2000 
    trace2ab = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)

unique, counts = np.unique(trace2ab["world_prior"], return_counts=True)
worldposterior_t1 = sort_category(unique, counts)
rmse_output_t1 = rmse(worldposterior_t1, worldhuman_emotion_t1)
print(rmse_output_t1)
unique, counts = np.unique(trace2ab["world"], return_counts=True)
worldposterior_t2 = sort_category(unique, counts)
rmse_output_t2 = rmse(worldposterior_t2, worldhuman_emotion_t2)
print(rmse_output_t2)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief1_random, world_prior, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 4 seconds.
The acceptance probability does not match the target. It is 0.7181823987179625, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


0.008793143141218023
0.033023137790958945


In [27]:
with pm.Model() as model2c:
    noise = .15
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=1 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    draw = 2000 
    trace2c = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)

unique, counts = np.unique(trace2c["world"], return_counts=True)
worldposterior_t3 = sort_category(unique, counts)
rmse_output_t3 = rmse(worldposterior_t3, worldhuman_emotion_t3)
print(rmse_output_t3)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 5 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.04104348105870343


In [28]:
with pm.Model() as model2d:
    noise = .15
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # bulb=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p)
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if belief both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p)
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=1 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_blue_p = pm.Deterministic("action2_blue_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,0)*(1-noise) + \
                                                                  # if belief both are needed and orange was pushed previously, then push blue
                                                                  tt.eq(belief2,2)*tt.eq(action1,1)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,1)*noise,
                                                                  # if outcome was reached at t1, no further action required
                                                                  noise)) 
    action2_blue_p = perturb(action2_blue_p)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_orange_p = pm.Deterministic("action2_orange_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,1)*(1-noise) + \
                                                                  # if belief both are needed and blue was pushed previously, then push orange 
                                                                  tt.eq(belief2,2)*tt.eq(action1,0)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,0)*noise,
                                                                  noise)) 
    action2_orange_p = perturb(action2_orange_p)
    action2_stack = tt.transpose(tt.stack(action2_blue_p, action2_orange_p, [noise*2]))
    action2 = pm.Categorical("action2", action2_stack, observed=1, shape=(1)) # blue orange 
    
    draw = 2000 
    trace2d = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace2d["world"], return_counts=True)
worldposterior_t4 = sort_category(unique, counts)
rmse_output_t4 = rmse(worldposterior_t4, worldhuman_emotion_t4)
print(rmse_output_t4)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 6 seconds.
The acceptance probability does not match the target. It is 0.7155314424812027, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


0.05124753488867856


In [29]:
with pm.Model() as model2:
    noise = .25
    world = pm.Categorical("world", [human_prior], shape=(1)) # blue, orange, both
    knowledge_p = pm.Beta("knowledge_p", a, b, shape=(1)) # reliability of the agent's knowledge
    knowledge = pm.Binomial("knowledge", 1, knowledge_p, shape=(1))
    
    belief1_random = pm.Categorical("belief1_random", [1,1,1], shape=(1))
    belief1 = pm.Deterministic("belief1", knowledge*world + (1-knowledge)*belief1_random)
    
    desire = pm.Binomial("desire",1, d, shape=(1)) # desire bulb to turn on=1, others=0

    action1_blue_p = pm.Deterministic("action1_blue_p", tt.eq(belief1,0)*desire*(1-noise) + \
                                      # if believe both are needed,
                                      tt.eq(belief1,2)*desire*(1-noise) + \
                                      # mistaken action
                                      tt.eq(belief1,1)*desire*noise +\
                                      # if desire something else
                                      (1-desire)*noise)
    action1_blue_p = perturb(action1_blue_p) # perturb ensures that the probability is not equals to 1 or 0
    
    action1_orange_p = pm.Deterministic("action1_orange_p", tt.eq(belief1,1)*desire*(1-noise) + \
                                        # if believe both are needed,
                                        tt.eq(belief1,2)*desire*(1-noise) + \
                                        # mistaken action
                                        tt.eq(belief1,0)*desire*noise +\
                                        # if desire something else,
                                        (1-desire)*noise)
    action1_orange_p = perturb(action1_orange_p) # perturb ensures that the probability is not equals to 1 or 0
    
    action1_stack = tt.transpose(tt.stack(action1_blue_p,action1_orange_p,[noise*2]))
    action1 = pm.Categorical("action1",action1_stack,observed=0,shape=(1)) # blue orange others
    
    outcome1_p = pm.Deterministic("outcome1_p", tt.eq(world,0)*tt.eq(action1,0)*(1-delay) + \
                                  tt.eq(world,1)*tt.eq(action1,1)*(1-delay) + \
                                  # if both boxes are required, then outcome will likely not happen
                                  tt.eq(world,2)*0)
    outcome1_p = perturb(outcome1_p,noise) # add noise to outcome
    
    outcome1 = pm.Binomial("outcome1", 1, outcome1_p, observed = 0 , shape = (1))
    
    happy1 = outcome1*desire*(1-noise) + noise*(1-desire)
    # if agent believes both actions are needed, they won't be frustrated at the lack of an outcome
    frustrated1 = (1-outcome1)*desire*tt.neq(belief1, 2)*(1-noise) + (1-outcome1)*desire*tt.eq(belief1, 2)*noise + noise*(1-desire)
    neutral1 = [1]
    emotion_array1 = tt.transpose(tt.stack(happy1, frustrated1, neutral1 ))
    expression1 = pm.Categorical("expression1",emotion_array1,observed=1 ) # happy, frustrated, neutral

    # if frustrated, agent is likely to revise belief
    # else stick to previous belief
    belief2_random = pm.Categorical("belief2_random", [1,1,1], shape=(1))
    belief2 = pm.Deterministic("belief2", tt.eq(expression1,1)*belief2_random + \
                              tt.neq(expression1,1)*belief1)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_blue_p = pm.Deterministic("action2_blue_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,0)*(1-noise) + \
                                                                  # if belief both are needed and orange was pushed previously, then push blue
                                                                  tt.eq(belief2,2)*tt.eq(action1,1)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,1)*noise,
                                                                  # if outcome was reached at t1, no further action required
                                                                  noise)) 
    action2_blue_p = perturb(action2_blue_p)
    
    # if outcome has been reached (and agent desired outcome), then likely no action would follow 
    action2_orange_p = pm.Deterministic("action2_orange_p", tt.switch(tt.eq(outcome1,0)*desire, \
                                                                  tt.eq(belief2,1)*(1-noise) + \
                                                                  # if believe both are needed and blue was pushed previously, then push orange 
                                                                  tt.eq(belief2,2)*tt.eq(action1,0)*(1-noise)+ \
                                                                  # mistaken action
                                                                  tt.eq(belief2,0)*noise,
                                                                  noise)) 
    action2_orange_p = perturb(action2_orange_p)
    action2_stack = tt.transpose(tt.stack(action2_blue_p, action2_orange_p, [noise*2]))
    action2 = pm.Categorical("action2", action2_stack, observed=1, shape=(1)) # blue orange others
    
    outcome2_p = pm.Deterministic("outcome2_p", tt.eq(world,0)*tt.eq(action2,0) + \
                                  # if previous action has a delayed effect
                                  tt.eq(world,0)*tt.eq(action1,0)*tt.neq(action2,0)*delay + \
                                  tt.eq(world,1)*tt.eq(action2,1)+ \
                                  # if previous action has a delayed effect
                                  tt.eq(world,1)*tt.eq(action1,1)*tt.neq(action2,1)*delay + \
                                  tt.eq(world,2)*tt.eq(action1,0)*tt.eq(action2,1)+ \
                                  tt.eq(world,2)*tt.eq(action1,1)*tt.eq(action2,0)) 
    outcome2_p = perturb(outcome2_p, noise)
    outcome2 = pm.Binomial("outcome2", 1, outcome2_p, observed = 1, shape = (1))
    
    happy2 = outcome2*desire*(1-noise) + noise*(1-desire)
    frustrated2 = (1-outcome2)*desire*(1-noise) + noise*(1-desire)
    neutral2 = [1]
    emotion_array2 = tt.transpose(tt.stack(happy2, frustrated2, neutral2 ))
    expression2 = pm.Categorical("expression2",emotion_array2,observed=0 ) # happy, frustrated, neutral
    
    draw = 2000 
    trace2e = pm.sample(draw, tune=1000, chains=4, return_inferencedata=False)
    
unique, counts = np.unique(trace2e["world"], return_counts=True)
worldposterior_t5 = sort_category(unique, counts)
rmse_output_t5 = rmse(worldposterior_t5, worldhuman_emotion_t5)
print(rmse_output_t5)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>CategoricalGibbsMetropolis: [belief2_random, belief1_random, world]
>NUTS: [knowledge_p]
>CompoundStep
>>Metropolis: [desire]
>>Metropolis: [knowledge]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 6 seconds.
The number of effective samples is smaller than 25% for some parameters.


0.09029890365596617


### Summary statistics

In [30]:
unique, counts = np.unique(trace2ab["world_prior"], return_counts=True)
print("world t1: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t1: {0: 3045, 1: 2454, 2: 2501}
[0.380625 0.30675  0.312625]


In [31]:
np.random.seed(123456)
print("world t1 :" + str(compute_ci_discrete(flatten(trace2ab["world_prior"]), HDI = .95, n = 1000, sim = 1000)))

world t1 :[array([0.352   , 0.278   , 0.281975]), array([0.41 , 0.335, 0.341])]


In [32]:
unique, counts = np.unique(trace2ab["world"], return_counts=True)
print("world t2: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t2: {0: 3792, 1: 1675, 2: 2533}
[0.474    0.209375 0.316625]


In [33]:
np.random.seed(123456)
print("world t2 :" + str(compute_ci_discrete(flatten(trace2ab["world"]), HDI = .95, n = 1000, sim = 1000)))

world t2 :[array([0.443, 0.185, 0.289]), array([0.506, 0.235, 0.347])]


In [34]:
unique, counts = np.unique(trace2c["world"], return_counts=True)
print("world t3: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t3: {0: 2214, 1: 2870, 2: 2916}
[0.27675 0.35875 0.3645 ]


In [35]:
np.random.seed(123456)
print("world t3 :" + str(compute_ci_discrete(flatten(trace2c["world"]), HDI = .95, n = 1000, sim = 1000)))

world t3 :[array([0.249, 0.33 , 0.336]), array([0.308   , 0.387025, 0.396   ])]


In [36]:
unique, counts = np.unique(trace2d["world"], return_counts=True)
print("world t4: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t4: {0: 2455, 1: 2729, 2: 2816}
[0.306875 0.341125 0.352   ]


In [37]:
np.random.seed(123456)
print("world t4 :" + str(compute_ci_discrete(flatten(trace2d["world"]), HDI = .95, n = 1000, sim = 1000)))

world t4 :[array([0.277975, 0.312975, 0.322   ]), array([0.336025, 0.375   , 0.383   ])]


In [38]:
unique, counts = np.unique(trace2e["world"], return_counts=True)
print("world t5: " + str(dict(zip(unique, counts))))
print(sort_category(unique, counts))

world t5: {0: 647, 1: 3677, 2: 3676}
[0.080875 0.459625 0.4595  ]


In [39]:
np.random.seed(123456)
print("world t5 :" + str(compute_ci_discrete(flatten(trace2e["world"]), HDI = .95, n = 1000, sim = 1000)))

world t5 :[array([0.064975, 0.426   , 0.429   ]), array([0.098, 0.489, 0.491])]
